For the purchase cost of equipment, the following code can be used to generate a dictionary

In [ ]:
import pandas as pd

purchase_df = pd.DataFrame({
    'Unit': [u.ID for u in my_sys.cost_units],                          # The system here is named my_sys
    'Purchase Cost': [u.purchase_cost for u in my_sys.cost_units]
})
purchase_df.to_excel('equipment_purchase_costs.xlsx', index=False)


To simply print these out, one can use:

In [ ]:
for u in my_sys.cost_units:
    print(f"{u.ID}: {u.purchase_cost}")

To generate code to display the utility consumption (both heating and cooling are specified) of unit operations and generate a dataframe

In [ ]:
import pandas as pd
from IPython.display import display

records = []

for unit in my_sys.units:    # system name is my_sys here
    heat_utilities = getattr(unit, 'heat_utilities', [])  # Default to empty list if not present
    total_duty = sum(h.duty for h in heat_utilities if h.duty != 0)  # Can also use h.duty > 0 for heating only

    records.append({
        'Unit ID': unit.ID,
        'Unit Type': type(unit).__name__,
        'Utility [kJ/hr]': total_duty
    })

df_opex = pd.DataFrame(records)

# Remove zero-utility rows (optional)
df_opex = df_opex[df_opex['Utility [kJ/hr]'] != 0]

# Sort in descending order of heat duty
df_opex = df_opex.sort_values(by='Utility [kJ/hr]', ascending=False).reset_index(drop=True)

total_heat = df_opex['Utility [kJ/hr]'].sum()
df_opex['% of Total'] = df_opex['Utility [kJ/bhr]'] / total_heat 
df_opex.to_excel('heat_utilities_summary.xlsx', index=False)

display(df_opex)

Generate code for total heating and cooling (this code is an extension of the above code if only total utility values are required)

In [ ]:

total_heating = df_opex[df_opex['Utility [kJ/hr]'] > 0]['Utility [kJ/hr]'].sum()
total_cooling= -df_opex[df_opex['Utility [kJ/hr]'] < 0]['Utility [kJ/hr]'].sum()

In [ ]:
import biosteam as bst
bst.Reaction

NameError: name 'bi' is not defined

The code below checks for a certain heating/cooling agent name (example shows natural gas) and searches through the system and finds out which unit operations use it and their duties

In [ ]:

def find_units_using_agent(agent_ID, system=None):
    agent = bst.settings.get_agent(agent_ID)
    if not agent:
        print(f"No agent found with ID '{agent_ID}'")
        return []

    if system is None:
        system = bst.main_flowsheet.my_sys    # Assynes systems name is my_sys

    units_using_agent = []

    for unit in system.units:
        if hasattr(unit, 'heat_utilities'):
            for hu in unit.heat_utilities:
                if hu.agent is agent:
                    units_using_agent.append((unit.ID, hu.duty, hu.agent.ID))

    return units_using_agent

# Example usage
results = find_units_using_agent('natural_gas')

# Display results
if results:
    print(f"\nUnits using agent 'ethylene':")
    for ID, duty, agent_ID in results:
        print(f" - Unit {ID} | Duty: {duty:.2f} kJ  | Agent: {agent_ID}")
else:
    print("No units are using this agent.")